## Columbia University
### ECBM E4040 Neural Networks and Deep Learning. Fall 2023.

## **Task 3: RNN Application -- Neural Machine Translation** (25%)

In this task, you are going to perform neural machine translation (NMT). NMT involves using a neural network to translate from one language to another. This is a widely studied natural language processing (NLP) problem and has tremendous real-world applications.

Machine Translation is a challenging task that involves both the usage of complex architectures and data processing tricks to obtain human-level performance. **In this notebook, you will implement a simple Seq2Seq architecture using RNN layers in keras.**

**The goal is to train a model to translate from Dutch (input language) to English (target language)**. This notebook uses data from the [Tab Delimited Bilingual Sentence Pairs](https://www.manythings.org/anki/) repository. You can find many such language pairs here.

## <span style="color:red"><strong>NOTE: Training this model may take 10-15 minutes of time depending on the strength of the system, so please plan accordingly.</strong></span>

In [12]:
# Import modules
import tensorflow as tf
import numpy as np
import json
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Broad Overview of Steps:
1. Preprocess and encode data
2. Create dataset/dataloaders
3. Define Model Architecture
4. Train Model
5. Evaluate results

Step 1. has already been completed for you. We provide two .npy files that contain the data: 
- `nmt_eng.npy` contains the encoded English sentences.
- `nmt_nl.npy` contains the encoded Dutch sentences. 
The sentences already have been normalized, padded and appended with the \<start\> and \<end\> tokens.

We also provide two vocabulary files `eng_vocab.txt` and `nl_vocab.txt` for the English and Dutch languages respectively. The vocabulary files will be used for decoding the input and output of our model.

## Part 1. Load Encoded Data

<font color="red"><strong>TODO:</strong></font> Execute the following cells to load the text data.

In [13]:
# Load Vocabulary files (dictionaries of word:int pairs)
with open("text_data/eng_vocab.txt", 'r') as f:
    eng_vocab = json.load(f)

with open("text_data/nl_vocab.txt", 'r') as f:
    nl_vocab = json.load(f)
    
eng_vocab = {int(key): value for key, value in eng_vocab.items()}
nl_vocab = {int(key): value for key, value in nl_vocab.items()}

print(f'Size of english vocab: {len(eng_vocab)}')
print(f'Size of dutch vocab: {len(nl_vocab)}')

Size of english vocab: 9044
Size of dutch vocab: 10000


In [14]:
# Load Encoded Sentence Data
eng_text = np.load("text_data/nmt_eng.npy")
nl_text = np.load("text_data/nmt_nl.npy")

print(f'Shape of english text data: {eng_text.shape}')
print(f'Shape of dutch text data: {nl_text.shape}')

Shape of english text data: (75298, 30)
Shape of dutch text data: (75298, 30)


## Part 2: Datasets and Dataloading (3%)

<font color="red"><strong>TODO:</strong></font> <b>Complete the functions in utils/translation/text_data.py</b>

This will create the train, validation, and test datasets for our translation model.

In [15]:
from utils.translation.text_data import get_dataset, get_dataset_partitions_tf, decode_text

text_ds = get_dataset(nl_text, eng_text)
train_ds, val_ds = get_dataset_partitions_tf(text_ds, len(text_ds))
print(f"Train size: {len(train_ds)}")
print(f"Validation size: {len(val_ds)}")

Train size: 67769
Validation size: 7529


In [16]:
# Let's have a look at a sample from the dataset
sample = next(iter(train_ds))
sample[0], sample[1]

(<tf.Tensor: shape=(30,), dtype=int64, numpy=
 array([  2,  18,  71,  17,   9,  36, 640,   7,   3,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0])>,
 <tf.Tensor: shape=(30,), dtype=int64, numpy=
 array([ 2, 62, 47,  6, 67, 13, 77,  8,  3,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])>)

In [17]:
decoded_nl = decode_text(sample[0].numpy(), vocab=nl_vocab)
decoded_eng = decode_text(sample[1].numpy(), vocab=eng_vocab)
print('NL:', decoded_nl)
print()
print('EN:', decoded_eng)

NL: ['[SOS]', 'van', 'wie', 'heb', 'je', 'dit', 'gekregen', '?', '[EOS]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']

EN: ['[SOS]', 'who', 'did', 'you', 'get', 'that', 'from', '?', '[EOS]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


## Part 3: Model Architecture (15%)

### Seq2Seq Model

In the general case, input sequences and output sequences have different lengths (e.g. machine translation) and the entire input sequence is required in order to start predicting the target. This requires a more advanced setup, which is what people commonly refer to when mentioning "sequence-to-sequence models" with no further context. Here's how it works (This example is English to French):

- An RNN layer (or stack thereof) acts as "encoder": it processes the input sequence and returns its own internal state. Note that we discard the outputs of the encoder RNN, only recovering the state. This state will serve as the "context", or "conditioning", of the decoder in the next step.
- Another RNN layer (or stack thereof) acts as "decoder": it is trained to predict the next characters of the target sequence, given previous characters of the target sequence. Specifically, it is trained to turn the target sequences into the same sequences but offset by one timestep in the future, a training process called "teacher forcing" in this context. Importantly, the encoder uses as initial state the state vectors from the encoder, which is how the decoder obtains information about what it is supposed to generate. Effectively, the decoder learns to generate targets[t+1...] given targets[...t], conditioned on the input sequence.

![teacher_forcing](./img/seq2seq-teacher-forcing.png)

In inference mode, i.e. when we want to decode unknown input sequences, we go through a slightly different process:

1) Encode the input sequence into state vectors.
2) Start with a target sequence of size 1 (just the start-of-sequence character).
3) Feed the state vectors and 1-char target sequence to the decoder to produce predictions for the next character.
4) Sample the next character using these predictions (we simply use argmax).
5) Append the sampled character to the target sequence
6) Repeat until we generate the end-of-sequence character or we hit the character limit.

![seq2seq-inference](./img/seq2seq-inference.png)

The seq2seq model implementation requires a more complex setup than what is provided by keras.Sequential().
You will be exposed to writing modular code using custom `keras.layer` and `keras.Model` classes. **First, please read https://keras.io/guides/making_new_layers_and_models_via_subclassing/** to get an idea about writing custom modules in tensorflow/keras, which is what is done in practice to implement complex architectures.

<font color="red"><strong>TODO:</strong></font> <b>Based on the above, you need to complete the code in utils/translation/layers.py</b>

In [18]:
#BATCH, PREFETCH, CACHE the datasets
#You can change the batch size based on memory requirements
BATCH_SIZE = 64
train_loader = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE).cache()
val_loader = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE).cache()

In [19]:
from utils.translation.layers import TranslationModel

eng_vocab_size = len(eng_vocab)
nl_vocab_size = len(nl_vocab)
hidden_size = 256

#Initialize Model
model = TranslationModel(nl_vocab_size, eng_vocab_size, hidden_size, eng_vocab)

## Part 4: Training the Model (5%)

The following cell(s) will train your Machine Translation model. The loss function used is Cross Entropy (since we are performing classification across the vocabulary at each time step). In practice, we usually implement a machine translation metric such as BLEU or ROUGE ([reference](https://medium.com/@sthanikamsanthosh1994/understanding-bleu-and-rouge-score-for-nlp-evaluation-1ab334ecadcb#:~:text=While%20BLEU%20score%20is%20primarily,the%20reference%20translations%20or%20summaries.)), and compute it for the validation set after each epoch. For this task, it is sufficient to just observe the train loss values.

You are already provided the `train_seq2seq_model` function in `utils.translation.train_funcs.py`. You can refer to this file to see the loss function and how a custom training loop with modifications has been implemented. Execute the cell below to train your model.

**Note that training will proceed as expected only if the implementation of your model is correct.** You can monitor the training loss to make sure that the model training is proceeding as expected. **Training may take 10-15 minutes depending on the strength of the system.**

If you have spare time, feel free to increase the number of epoch gauge if the performance improves. 

In [20]:
from utils.translation.train_funcs import train_seq2seq_model

# Train the model. Use the Adam optimizer with 1e-3 learning rate.
num_epochs = 5
learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

train_seq2seq_model(
    model,
    train_loader,
    optimizer,
    num_epochs
)


Epoch: 1/5
Iter: 0, Loss (iter): 9.109655380249023, Mean Loss (over last 50 iters): 9.109655380249023
Iter: 50, Loss (iter): 5.561922073364258, Mean Loss (over last 50 iters): 6.438636302947998
Iter: 100, Loss (iter): 5.214922904968262, Mean Loss (over last 50 iters): 5.5021772384643555
Iter: 150, Loss (iter): 5.426918029785156, Mean Loss (over last 50 iters): 5.2833685874938965
Iter: 200, Loss (iter): 4.98797607421875, Mean Loss (over last 50 iters): 5.141422271728516
Iter: 250, Loss (iter): 4.985249042510986, Mean Loss (over last 50 iters): 5.048381805419922
Iter: 300, Loss (iter): 4.848669528961182, Mean Loss (over last 50 iters): 4.949271202087402
Iter: 350, Loss (iter): 4.706375598907471, Mean Loss (over last 50 iters): 4.819245338439941
Iter: 400, Loss (iter): 4.440303802490234, Mean Loss (over last 50 iters): 4.677301406860352
Iter: 450, Loss (iter): 4.419108867645264, Mean Loss (over last 50 iters): 4.612499713897705
Iter: 500, Loss (iter): 4.456099510192871, Mean Loss (over la

Iter: 1050, Loss (iter): 2.136608600616455, Mean Loss (over last 50 iters): 2.1133933067321777
Epoch: 5/5
Iter: 0, Loss (iter): 2.31770920753479, Mean Loss (over last 50 iters): 2.1116552352905273
Iter: 50, Loss (iter): 2.0536561012268066, Mean Loss (over last 50 iters): 2.11337947845459
Iter: 100, Loss (iter): 1.9534906148910522, Mean Loss (over last 50 iters): 2.0997748374938965
Iter: 150, Loss (iter): 2.2496261596679688, Mean Loss (over last 50 iters): 2.047212839126587
Iter: 200, Loss (iter): 1.979124903678894, Mean Loss (over last 50 iters): 2.0501534938812256
Iter: 250, Loss (iter): 1.8822513818740845, Mean Loss (over last 50 iters): 2.012162923812866
Iter: 300, Loss (iter): 2.057410478591919, Mean Loss (over last 50 iters): 2.0132553577423096
Iter: 350, Loss (iter): 2.055738925933838, Mean Loss (over last 50 iters): 1.986190915107727
Iter: 400, Loss (iter): 1.94454026222229, Mean Loss (over last 50 iters): 1.9567152261734009
Iter: 450, Loss (iter): 1.735944390296936, Mean Loss (

## Part 5: Evaluating Results (2%)

Our training function only shows the training loss value. To assess the performance of the model, we can perform some predictions and decode the input/output sentences. 

<font color="red"><strong>TODO:</strong></font> Run the following cells to qualitatively asses the quality of the generated sentences and the performance of the trained model.

**NOTE**: As we are dealing with a generation task, the outputs will vary depending on the final trained model. Therefore, we have provided a set of example outputs with the translation quality you can expect from the trained model. You results may be different.



In [21]:
# Run these cells to evaluate your model on one batch of the validation set
val_sample = val_loader.shuffle(10000).take(1)
val_sample = next(iter(val_sample))

In [22]:
val_inp, val_target = val_sample
decoded_inputs = []
reserved_tokens = ['[PAD]', '[SOS]', '[EOS]']
for inp in val_inp.numpy():
    decoded_text = decode_text(inp, vocab=nl_vocab)
    decoded_inputs.append([token for token in decoded_text if token not in reserved_tokens])

val_pred = model(val_inp, training=False)
decoded_outputs = model.decode_tokens(val_pred)
decoded_ground_truth = model.decode_tokens(val_target[:, 1:])

samples_to_show = 10 #Should be <= batch size

for i, decoded_data in enumerate(zip(decoded_inputs, decoded_ground_truth, decoded_outputs)):
    nl_sentence = ' '.join(decoded_data[0])
    gt_en_sentence = ' '.join(decoded_data[1])
    pred_en_sentence = ' '.join(decoded_data[2])
    print('Sample:', i+1)
    print('Dutch Sentence: ', nl_sentence)
    print('English Sentence (Truth): ', gt_en_sentence)
    print('English Sentence (Pred)', pred_en_sentence)
    if i > samples_to_show - 1:
        break

Sample: 1
Dutch Sentence:  vanwege haar hulp verbeterde mijn frans beetje bij beetje
English Sentence (Truth):  with her help my french improved little by little
English Sentence (Pred) mom and i have to speak english with my parents
Sample: 2
Dutch Sentence:  morgen blijf ik thuis
English Sentence (Truth):  i ll stay home tomorrow
English Sentence (Pred) i ll stay at home tomorrow
Sample: 3
Dutch Sentence:  ik maak een sneeuwpop
English Sentence (Truth):  i m making a snowman
English Sentence (Pred) i m a [UNK]
Sample: 4
Dutch Sentence:  ze hebben me verteld om hier niet te komen
English Sentence (Truth):  i was told not to come here
English Sentence (Pred) they told me not to be here
Sample: 5
Dutch Sentence:  ze stond voor de klas
English Sentence (Truth):  she used to be a teacher
English Sentence (Pred) she was in the hospital
Sample: 6
Dutch Sentence:  ik heb geprobeerd uw kantoor te bellen
English Sentence (Truth):  i tried calling your office
English Sentence (Pred) i ve never 

### Example of Expected Outputs

In [ ]:
"""
Sample: 1
Dutch Sentence:  waar hebben jullie het verstopt ?
English Sentence (Truth):  where did you hide it ?
English Sentence (Pred) where did you do it ?
Sample: 2
Dutch Sentence:  denk jij dit ?
English Sentence (Truth):  is that what you think ?
English Sentence (Pred) do you think this ?
Sample: 3
Dutch Sentence:  de treinen rijden s nachts minder vaak
English Sentence (Truth):  the trains don t run as often at night
English Sentence (Pred) the [UNK] [UNK] to [UNK] a week
Sample: 4
Dutch Sentence:  ik weet hoe dit werkt
English Sentence (Truth):  i know how this works
English Sentence (Pred) i know that this dictionary
Sample: 5
Dutch Sentence:  houd je toespraak kort
English Sentence (Truth):  keep your speech short
English Sentence (Pred) look at your country
Sample: 6
Dutch Sentence:  help me alsjeblieft een trui uit te kiezen die bij mijn nieuwe jurk past
English Sentence (Truth):  please help me pick out a sweater which matches my new dress
English Sentence (Pred) please give me a new dictionary for me this year ago
Sample: 7
Dutch Sentence:  welk jaar is het ?
English Sentence (Truth):  what year is it ?
English Sentence (Pred) what is the last ?
Sample: 8
Dutch Sentence:  welk verschil is er tussen dit en dat ?
English Sentence (Truth):  what is the difference between this and that ?
English Sentence (Pred) what s the difference between this bird ?
Sample: 9
Dutch Sentence:  is dat onze bus ?
English Sentence (Truth):  is that our bus ?
English Sentence (Pred) is this the book ?
Sample: 10
Dutch Sentence:  kun je me vannacht een [UNK] doen en op mijn kinderen oppassen ?
English Sentence (Truth):  could you do me a favor and [UNK] my kids tonight ?
English Sentence (Pred) can you please tell the truth to me a doctor ?
Sample: 11
Dutch Sentence:  ik bleef daar
English Sentence (Truth):  i stayed there
English Sentence (Pred) i felt [UNK]
"""

### <font color="red"><strong>TODO:</strong></font> <b>Answer the following questions:</b>

1. **Describe your observations of the model's evaluation performance. Briefly explain any one method to improve the model architecture based on the lecture readings, or online sources.**

<span style="color:red">__Answer:__</span>
The model shows mixed accuracy in translating Dutch to English, effectively handling some sentences but struggling with complex structures and specific vocabulary. Notable are the errors and misinterpretations in several samples, indicating difficulties in capturing nuances and dealing with unknown words. To improve, integrating an attention mechanism is advisable. This approach helps the model focus on relevant parts of the input for each word in the output, particularly beneficial for longer sentences. Implementing attention requires adding a specialized layer to the neural network and adjusting both encoder and decoder components. Overall, while capable of basic translations, the model needs enhancements, particularly in handling complex language elements, which attention mechanisms can address.



2. **During the data preprocessing, we encoded each word in the input/target language to a number based on the vocabulary. This is known as tokenization. Briefly explain any one other method of tokenization, and why it might be beneficial to this particular task.**

<span style="color:red">__Answer:__</span>
Byte Pair Encoding (BPE): A popular subword tokenization method, initially used for data compression, that iteratively merges frequent character pairs to form common subwords.

Byte Pair Encoding (BPE) is highly beneficial for machine translation as it efficiently manages rare and unknown words by breaking them into known subword units. This approach allows models to handle unfamiliar vocabulary, crucial in translation tasks. BPE balances vocabulary size with language diversity, accommodating inflected languages by representing various word forms without needing an exhaustive vocabulary. It effectively integrates loanwords, proper nouns, and technical terms by breaking them into processable subwords. Additionally, BPE's ability to uncover common subwords across languages enhances translation quality, especially for linguistically related languages, thereby improving the model's overall flexibility and robustness in handling natural language nuances.



We implemented a simple LSTM-based seq2seq model. The performance may not be the best, since Dutch and English are naturally complex languages. The state-of-the-art translation models are based on Transformer Networks that use the attention mechanism. (further reading: https://nlpprogress.com/english/machine_translation.html)

## (BONUS) Part 6: Bidirectional LSTM (5%)

One simple modification that we can do to significantly improve the quality of the generated sentences is to change the Encoder LSTM to be bidirectional. This will improve the performance because different languages tend to have different sentence structures, and in the case of Dutch, crucial information for a given word may not be available until later on in the sentence. 

**SIDE NOTE**: Changing the decoder to be bidirectional will not work in a text generation task (in our case, translation). Feel free to think of why this is the case. (No written is required).

<font color="red"><strong>BONUS TODO:</strong></font> <b> Complete the BidirectionalEncoder Class in utils/translation/layers.py, and run the training and validation loops to compare the generated translations with the previous model you implemented.</b>

In [23]:
from utils.translation.layers import TranslationModel

eng_vocab_size = len(eng_vocab)
nl_vocab_size = len(nl_vocab)
hidden_size = 256

# Initialize Model with bidirectional_encoder = True
# NOTE the bidirectional_encoder = True
model = TranslationModel(nl_vocab_size, eng_vocab_size, hidden_size, eng_vocab, bidirectional_encoder=True)

In [24]:
from utils.translation.train_funcs import train_seq2seq_model

# Train the model. Use the Adam optimizer with 1e-3 learning rate.
num_epochs = 5
learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

train_seq2seq_model(
    model,
    train_loader,
    optimizer,
    num_epochs
)

Epoch: 1/5
Iter: 0, Loss (iter): 9.109925270080566, Mean Loss (over last 50 iters): 9.109925270080566
Iter: 50, Loss (iter): 5.590933799743652, Mean Loss (over last 50 iters): 6.434598445892334
Iter: 100, Loss (iter): 5.217302322387695, Mean Loss (over last 50 iters): 5.515486240386963
Iter: 150, Loss (iter): 5.433877468109131, Mean Loss (over last 50 iters): 5.275567054748535
Iter: 200, Loss (iter): 4.962477684020996, Mean Loss (over last 50 iters): 5.11947774887085
Iter: 250, Loss (iter): 4.911623954772949, Mean Loss (over last 50 iters): 4.987344741821289
Iter: 300, Loss (iter): 4.743401527404785, Mean Loss (over last 50 iters): 4.850338935852051
Iter: 350, Loss (iter): 4.550870895385742, Mean Loss (over last 50 iters): 4.684281826019287
Iter: 400, Loss (iter): 4.284757137298584, Mean Loss (over last 50 iters): 4.5463056564331055
Iter: 450, Loss (iter): 4.201324462890625, Mean Loss (over last 50 iters): 4.4350690841674805
Iter: 500, Loss (iter): 4.21593713760376, Mean Loss (over las

Iter: 1050, Loss (iter): 1.9437633752822876, Mean Loss (over last 50 iters): 1.8428921699523926
Epoch: 5/5
Iter: 0, Loss (iter): 2.0554192066192627, Mean Loss (over last 50 iters): 1.846621036529541
Iter: 50, Loss (iter): 1.7325129508972168, Mean Loss (over last 50 iters): 1.8505859375
Iter: 100, Loss (iter): 1.6618834733963013, Mean Loss (over last 50 iters): 1.8362765312194824
Iter: 150, Loss (iter): 1.9841867685317993, Mean Loss (over last 50 iters): 1.7974162101745605
Iter: 200, Loss (iter): 1.8311561346054077, Mean Loss (over last 50 iters): 1.7903605699539185
Iter: 250, Loss (iter): 1.628451943397522, Mean Loss (over last 50 iters): 1.7640877962112427
Iter: 300, Loss (iter): 1.8788560628890991, Mean Loss (over last 50 iters): 1.7517120838165283
Iter: 350, Loss (iter): 1.8563867807388306, Mean Loss (over last 50 iters): 1.742863416671753
Iter: 400, Loss (iter): 1.648897409439087, Mean Loss (over last 50 iters): 1.7100034952163696
Iter: 450, Loss (iter): 1.5147981643676758, Mean Lo

In [25]:
# Run these cells to evaluate your model on one batch of the validation set
val_sample = val_loader.shuffle(10000).take(1)
val_sample = next(iter(val_sample))

In [26]:
val_inp, val_target = val_sample
decoded_inputs = []
reserved_tokens = ['[PAD]', '[SOS]', '[EOS]']
for inp in val_inp.numpy():
    decoded_text = decode_text(inp, vocab=nl_vocab)
    decoded_inputs.append([token for token in decoded_text if token not in reserved_tokens])

val_pred = model(val_inp, training=False)
decoded_outputs = model.decode_tokens(val_pred)
decoded_ground_truth = model.decode_tokens(val_target[:, 1:])

samples_to_show = 10 #Should be <= batch size

for i, decoded_data in enumerate(zip(decoded_inputs, decoded_ground_truth, decoded_outputs)):
    nl_sentence = ' '.join(decoded_data[0])
    gt_en_sentence = ' '.join(decoded_data[1])
    pred_en_sentence = ' '.join(decoded_data[2])
    print('Sample:', i+1)
    print('Dutch Sentence: ', nl_sentence)
    print('English Sentence (Truth): ', gt_en_sentence)
    print('English Sentence (Pred)', pred_en_sentence)
    if i > samples_to_show - 1:
        break

Sample: 1
Dutch Sentence:  ik heb ook in boston gewoond
English Sentence (Truth):  i ve also lived in boston
English Sentence (Pred) i lived in boston
Sample: 2
Dutch Sentence:  of je het nu leuk vindt of niet je moet gaan
English Sentence (Truth):  like it or not you have to go
English Sentence (Pred) you don t feel better to do that
Sample: 3
Dutch Sentence:  hij hoeft het niet te weten
English Sentence (Truth):  he doesn t have to know
English Sentence (Pred) he doesn t want to do it
Sample: 4
Dutch Sentence:  tom checkte zijn e mails
English Sentence (Truth):  tom checked his emails
English Sentence (Pred) tom checked his email
Sample: 5
Dutch Sentence:  het was dringend
English Sentence (Truth):  it was urgent
English Sentence (Pred) it was snowing
Sample: 6
Dutch Sentence:  wat fijn om je weer te zien
English Sentence (Truth):  how nice to see you again
English Sentence (Pred) what kind of things do you like to see
Sample: 7
Dutch Sentence:  je kunt nu beter weggaan
English Sente

<font color="red"><strong>BONUS TODO:</strong></font> <b> Briefly describe the differences in quality of the translations between the model with and without a bidirectional encoder. (1-2 sentenes is enough) <b/>

<span style="color:red">__Answer:__</span>
The model with a bidirectional encoder generally produces higher quality translations compared to the model without it, as the bidirectional encoder captures contextual information from both directions of the input sequence, leading to a more nuanced and accurate understanding of the text. This results in translations that are more coherent and contextually appropriate.
